# Simpler RAG Chatbot Model
This notebook implements a Retrieval-Augmented Generation (RAG) approach using the `llama_index` library. It reads documents from a directory, creates a vector index, sets up a query engine, and processes queries. The index is persisted for later use.


In [2]:
# Install required packages.
!pip install -r simpler_rag_requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'simpler_rag_requirements.txt'

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Environment Setup
This cell imports required libraries, loads environment variables (such as the OpenAI API key) from a `.env` file, and sets the `OPENAI_API_KEY` in the environment.


In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file.
load_dotenv()

# Set the OPENAI_API_KEY environment variable from the loaded variables.
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


## Load Documents and Build the Initial Index
This cell uses `SimpleDirectoryReader` from `llama_index` to load documents from the "data" folder and creates a vector index from these documents. The index is built with a progress indicator.


In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Load documents from the "data" directory.
documents = SimpleDirectoryReader("data").load_data()
print(documents)

# Create a vector index from the loaded documents with progress shown.
index = VectorStoreIndex.from_documents(documents, show_progress=True)
print(index)

# Create a basic query engine from the index.
query_engine = index.as_query_engine()


[Document(id_='e2ee37c8-5f0f-4380-a774-16d53c1cdd5f', embedding=None, metadata={'page_label': '1', 'file_name': 'standard-residential-lease-agreement-template.pdf', 'file_path': 'data\\standard-residential-lease-agreement-template.pdf', 'file_type': 'application/pdf', 'file_size': 144887, 'creation_date': '2025-03-03', 'last_modified_date': '2025-03-02', 'last_accessed_date': '2025-03-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='STANDARD LEASE AGREEMENT\n\xa0\n\xa0\nTHIS LEASE AGREEMENT hereinafter known as the "Lease" is made and entered into\n\xa0\nthis ____ day of _______________________,  20____, by and between the Landlord known\n\xa0\nas _______________________ with a mailing address _______________________, in the City\n\xa0\nof _________

c:\Users\amaan\Desktop\RAG Chatbot\basic_rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 8/8 [00:01<00:00,  5.50it/s]


## Setup Advanced Query Engine Components
In this cell, we import additional components for retrieval and postprocessing. We then create a retriever that uses the vector index (returning the top 4 similar documents) and a postprocessor to filter out candidates with low similarity. These are used to set up an advanced query engine.


In [5]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

# Create a retriever using the vector index (top 4 results).
retriever = VectorIndexRetriever(index=index, similarity_top_k=4)

# Set up a postprocessor with a similarity cutoff of 0.80.
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.80)

# Create an advanced query engine that applies the retriever and postprocessor.
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[postprocessor]
)


## Persisting the Index
This cell checks if a persisted index already exists in the "./storage" directory. If not, it creates the index from documents and saves it; otherwise, it loads the index from storage. This allows faster startup on subsequent runs.


In [6]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# Directory where the index is persisted.
PERSIST_DIR = "./storage"

if not os.path.exists(PERSIST_DIR):
    # If no storage exists, load documents and create the index.
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # Persist the index for future use.
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # If storage exists, load the index from the persisted storage.
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)


## Querying the Index
Finally, this cell converts the index into a query engine and processes a sample query. The response is printed to the output.


In [7]:
# Create a query engine from the index.
query_engine = index.as_query_engine()

# Execute a sample query.
response = query_engine.query("What is the stance on retaliatory acts?")
print(response)


The Landlord is prohibited from making any type of retaliatory acts against the Tenant(s), including but not limited to restricting access to the Property, decreasing or canceling services or utilities, failure to repair appliances or fixtures, or any other type of activity that could be considered unjustified.
